In [1]:
import os
import math
import cv2
import numpy as np
import utils
import tensorflow as tf
import tensorflow.contrib.slim as slim
from libs.networks.network_factory import get_network_byname
from libs import build_rpn, build_fast_rcnn, build_fpn
from data.io.read_tfrecord import eval_predict_input_fn
from configs.config import Config
from libs.visualize import display_instances
from libs.label_dict import LABEl_NAME_MAP
from tools.test import model_fn

%matplotlib inline 

####  Define the evaluate function

In [2]:
def compute_match(gt_boxes, gt_class_ids,
               pred_boxes, pred_class_ids, pred_scores,
               iou_threshold=0.5):
    """Compute Average Precision at a set IoU threshold (default 0.5).

    Returns:
    gt_match_dict: array for gt has been preded(1 be for has been pred, 0 for don't been pred) 
    pred_match_dict: array for pred is or not right(1 is right, 0 is wrong)
    pred_scores_dict: array for has been sorted.(be correspend to pred_match)

    """
    gt_match_dict = {}
    pred_match_dict = {}
    pred_scores_dict = {}
    # Trim zero padding and sort predictions by score from high to low
    # TODO: cleaner to do zero unpadding upstream
    gt_boxes = utils.trim_zeros(gt_boxes)
    pred_boxes = utils.trim_zeros(pred_boxes)
    pred_scores = pred_scores[:pred_boxes.shape[0]]
    indices = np.argsort(pred_scores)[::-1]
    pred_boxes = pred_boxes[indices]
    pred_class_ids = pred_class_ids[indices]
    pred_scores = pred_scores[indices]
    
    # Compute IoU overlaps [pred_boxes, gt_boxes]
    overlaps = utils.compute_overlaps(pred_boxes, gt_boxes)
    
    # Loop through ground truth boxes and find matching predictions
    match_count = 0
    pred_match = np.zeros([pred_boxes.shape[0]])
    gt_match = np.zeros([gt_boxes.shape[0]])
    for i in range(len(pred_boxes)):
        # Find best matching ground truth box
        sorted_ixs = np.argsort(overlaps[i])[::-1]
        for j in sorted_ixs:
            # If ground truth box is already matched, go to next one
            if gt_match[j] == 1:
                continue
            # If we reach IoU smaller than the threshold, end the loop
            iou = overlaps[i, j]
            if iou < iou_threshold:
                break
            # Do we have a match?
            if pred_class_ids[i] == gt_class_ids[j]:
                match_count += 1
                gt_match[j] = 1
                pred_match[i] = 1
                break
    for class_id in range(1,12):
        gt_match_dict[class_id] =  gt_match[gt_class_ids==class_id]
        pred_match_dict[class_id] = pred_match[pred_class_ids==class_id]
        pred_scores_dict[class_id] = pred_scores[pred_class_ids==class_id]
    return gt_match_dict, pred_match_dict, pred_scores_dict

def compute_class_mAP(single_record):
    """
    single_record: the single result of model(it is a generator)
    
    all_class_mAP(array): all class AP.
    """
    
    all_gt_match_dict = {}
    all_pred_match_dict = {}
    all_pred_scores_dict = {}
    for class_id in range(1,12):
        all_gt_match_dict[class_id] = np.array([])
        all_pred_match_dict[class_id]= np.array([])
        all_pred_scores_dict[class_id]= np.array([])
        
    while True:
        # Load
        try:
            np_single_sample = next(single_record)
            predict_bbox = np.round(np_single_sample["predict_bbox"]) 
            predict_class_id = np_single_sample["predict_class_id"].astype(np.int32)
            predict_scores = np_single_sample["predict_scores"]
            gt_bbox = np_single_sample["gt_box_labels"][:, :4]
            gt_class_id = np_single_sample["gt_box_labels"][:, 4]
            # Run object detection
            # Compute AP
            gt_match_dict, pred_match_dict, pred_scores_dict =\
                compute_match(gt_bbox, gt_class_id,
                              predict_bbox, predict_class_id,
                              predict_scores)
            for class_id in range(1,12):
                all_gt_match_dict[class_id] = np.concatenate([all_gt_match_dict[class_id],gt_match_dict[class_id]])
                all_pred_match_dict[class_id]= np.concatenate([all_pred_match_dict[class_id],pred_match_dict[class_id]])
                all_pred_scores_dict[class_id]= np.concatenate([all_pred_scores_dict[class_id],pred_scores_dict[class_id]])
        except:
            break
            
    all_class_mAP = np.zeros([11])
    all_class_AR = np.zeros([11])
    for class_id in range(1,12):
        gt_match = all_gt_match_dict[class_id]
        pred_match = all_pred_match_dict[class_id]
        pred_score = all_pred_scores_dict[class_id]
        indices = np.argsort(pred_score)[::-1]
        pred_scores = pred_score[indices]
        pred_match = pred_match[indices]
        # Compute precision and recall at each prediction box step
        precisions = np.cumsum(pred_match) / (np.arange(len(pred_match)) + 1)
        recalls = np.cumsum(pred_match).astype(np.float32) / len(gt_match)

        # Pad with start and end values to simplify the math
        precisions = np.concatenate([[0], precisions, [0]])
        recalls = np.concatenate([[0], recalls, [1]])

        # Ensure precision values decrease but don't increase. This way, the
        # precision value at each recall threshold is the maximum it can be
        # for all following recall thresholds, as specified by the VOC paper.
        for i in range(len(precisions) - 2, -1, -1):
            precisions[i] = np.maximum(precisions[i], precisions[i + 1])

        # Compute mean AP over recall range
        indices = np.where(recalls[:-1] != recalls[1:])[0] + 1
        mAP = np.sum((recalls[indices] - recalls[indices - 1]) *
                     precisions[indices])
        all_class_mAP[class_id-1] = mAP
        all_class_AR[class_id-1] = recalls[-2]
    return all_class_mAP, all_class_AR

### Define prediction generator 

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
class PConfig(Config):
    PER_GPU_IMAGE = 1

net_config = PConfig()
session_config = tf.ConfigProto()
session_config.gpu_options.allow_growth = True
session_config.allow_soft_placement = True
estimator_config = tf.estimator.RunConfig(model_dir=net_config.MODLE_DIR,
                                          session_config=session_config)

my_estimator = tf.estimator.Estimator(model_fn,
                                      params={"net_config": net_config}, 
                                      config=estimator_config)
single_record = my_estimator.predict(input_fn=lambda: 
                                     eval_predict_input_fn(net_config.DATA_DIR),
                                     yield_single_examples=True)

INFO:tensorflow:Using config: {'_model_dir': './logs', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  allow_growth: true
}
allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0b7643c160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [4]:
all_mAP, all_AR = compute_class_mAP(single_record)

INFO:tensorflow:Calling model_fn.


/root/userfolder/anaconda3/lib/python3.6/site-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/root/userfolder/anaconda3/lib/python3.6/site-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./logs/model.ckpt-113179
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [5]:
import pandas as pd
class_names = LABEl_NAME_MAP
all_mAP_add = np.concatenate([all_mAP,np.array([np.mean(all_mAP)])])
allclass_mAP = pd.DataFrame(all_mAP_add,index=[class_names[i] for i in range(1,12)] + ["mAP"])
display(allclass_mAP)

,0
ascus,0.273919
asch,0.067316
lsil,0.417430
hsil,0.400730
scc,0.217877
agc,0.545329
trichomonas,0.449804
candida,0.655251
flora,0.635147
herps,0.680915


In [6]:
class_names = LABEl_NAME_MAP
all_AR_add = np.concatenate([all_AR, np.array([np.mean(all_AR)])])
allclass_mAR = pd.DataFrame(all_AR_add,index=[class_names[i] for i in range(1,12)] + ["mAR"])
display(allclass_mAR)

,0
ascus,0.556604
asch,0.194030
lsil,0.539394
hsil,0.685530
scc,0.412281
agc,0.738132
trichomonas,0.714885
candida,0.769231
flora,0.791667
herps,0.771429
